# **Homework 10 - Adversarial Attack**

## Enviroment & Download

We make use of [pytorchcv](https://pypi.org/project/pytorchcv/) to obtain CIFAR-10 pretrained model, so we need to set up the enviroment first. We also need to download the data (200 images) which we want to attack.

In [1]:
'''
# set up environment
!pip install pytorchcv
!pip install imgaug

# download
#!gdown --id 1t2UFQXr1cr5qLMBK2oN2rY1NDypi9Nyw --output data.zip

# if the above link isn't available, try this one

!wget https://www.dropbox.com/s/lbpypqamqjpt2qz/data.zip

# unzip
!unzip ./data.zip
'''


"\n# set up environment\n!pip install pytorchcv\n!pip install imgaug\n\n# download\n#!gdown --id 1t2UFQXr1cr5qLMBK2oN2rY1NDypi9Nyw --output data.zip\n\n# if the above link isn't available, try this one\n\n!wget https://www.dropbox.com/s/lbpypqamqjpt2qz/data.zip\n\n# unzip\n!unzip ./data.zip\n"

In [2]:

!rm ./data.zip

rm: cannot remove './data.zip': No such file or directory


In [3]:
import torch
import torch.nn as nn
from pytorchcv.model_provider import get_model as ptcv_get_model
import random
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 8

def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0) 

## Global Settings 
#### **[NOTE]**: Don't change the settings here, or your generated image might not meet the constraint.
* $\epsilon$ is fixed to be 8. But on **Data section**, we will first apply transforms on raw pixel value (0-255 scale) **by ToTensor (to 0-1 scale)** and then **Normalize (subtract mean divide std)**. $\epsilon$ should be set to $\frac{8}{255 * std}$ during attack.

* Explaination (optional)
    * Denote the first pixel of original image as $p$, and the first pixel of adversarial image as $a$.
    * The $\epsilon$ constraints tell us $\left| p-a \right| <= 8$.
    * ToTensor() can be seen as a function where $T(x) = x/255$.
    * Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.
    * After applying ToTensor() and Normalize() on $p$ and $a$, the constraint becomes $\left| N(T(p))-N(T(a)) \right| = \left| \frac{\frac{p}{255}-mean}{std}-\frac{\frac{a}{255}-mean}{std} \right| = \frac{1}{255 * std} \left| p-a \right| <= \frac{8}{255 * std}.$
    * So, we should set $\epsilon$ to be $\frac{8}{255 * std}$ after ToTensor() and Normalize().

In [4]:
# the mean and std are the calculated statistics from cifar_10 dataset
cifar_10_mean = (0.491, 0.482, 0.447) # mean for the three channels of cifar_10 images
cifar_10_std = (0.202, 0.199, 0.201) # std for the three channels of cifar_10 images

# convert mean and std to 3-dimensional tensors for future operations
mean = torch.tensor(cifar_10_mean).to(device).view(3, 1, 1)
std = torch.tensor(cifar_10_std).to(device).view(3, 1, 1)

epsilon = 8/255/std

In [5]:
root = './data' # directory for storing benign images
# benign images: images which do not contain adversarial perturbations
# adversarial images: images which include adversarial perturbations

## Data

Construct dataset and dataloader from root directory. Note that we store the filename of each image for future usage.

In [6]:
import os
import glob
import shutil
import numpy as np
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar_10_mean, cifar_10_std)
])

class AdvDataset(Dataset):
    def __init__(self, data_dir, transform):
        self.images = []
        self.labels = []
        self.names = []
        '''
        data_dir
        ├── class_dir
        │   ├── class1.png
        │   ├── ...
        │   ├── class20.png
        '''
        for i, class_dir in enumerate(sorted(glob.glob(f'{data_dir}/*'))):
            images = sorted(glob.glob(f'{class_dir}/*'))
            self.images += images
            self.labels += ([i] * len(images))
            self.names += [os.path.relpath(imgs, data_dir) for imgs in images]
        self.transform = transform
    def __getitem__(self, idx):
        image = self.transform(Image.open(self.images[idx]))
        label = self.labels[idx]
        return image, label
    def __getname__(self):
        return self.names
    def __len__(self):
        return len(self.images)

adv_set = AdvDataset(root, transform=transform)
adv_names = adv_set.__getname__()
adv_loader = DataLoader(adv_set, batch_size=batch_size, shuffle=False)

print(f'number of images = {adv_set.__len__()}')

number of images = 200


## Utils -- Benign Images Evaluation

In [7]:
# to evaluate the performance of model on benign images
def epoch_benign(model, loader, loss_fn):
    model.eval()
    train_acc, train_loss = 0.0, 0.0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        yp = model(x)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
    return train_acc / len(loader.dataset), train_loss / len(loader.dataset)

## Utils -- Attack Algorithm

In [8]:
import random
# perform fgsm attack
epsilon = 8/255/std
def fgsm(model, x, y, loss_fn, epsilon=epsilon):
    #global epsilon
    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
    loss = loss_fn(model(x_adv), y) # calculate loss
    loss.backward() # calculate gradient
    # fgsm: use gradient ascent on x_adv to maximize loss
    grad = x_adv.grad.detach()
    x_adv = x_adv + epsilon * grad.sign()
    return x_adv

# alpha and num_iter can be decided by yourself
alpha = 0.8/255/std
alpha = 2/255/std

def DIM(x):
  p=random.random()
  ratio=random.randint(25,31)
  #print(x.size())
  resize=random.randint(25,31)
  if p>=0.5:
    r1=random.randint(1,32-resize)
    l=32-r1
    r1=random.randint(1,32-resize)
    u=32-r1
    r=32-resize-l
    d=32-resize-u
    padding=(l,u,r,d)
    transform = transforms.Compose([
      transforms.Resize(resize),
      transforms.Pad(padding, fill=0,padding_mode="constant"), 
    ])
    x=transform(x)
    #print(x.size())

  
  return x
  
def ifgsm(model, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=20):
    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    #x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
    g_t = loss_fn(model(x_adv), y) # calculate loss

    min_adv,max_adv=torch.min(x_adv),torch.max(x_adv)
    #loss.backward() # calculate gradient
    # fgsm: use gradient ascent on x_adv to maximize loss
    #grad = x_adv.grad.detach()
    #x_adv = x_adv + epsilon * grad.sign()

    # initialze momentum tensor
    momentum = torch.zeros_like(x).detach().to(device)
    mu=0
    ################ TODO: Strong baseline ####################
    for i in range(num_iter):
      #print(i)
      # TODO: Refer to the algorithm of MI-FGSM
      # Calculate the momentum and update
      #x_adv=DIM(x_adv)
      #print(type(x_adv))
      x_adv.requires_grad = True
      #min_adv,max_adv=torch.min(x_adv),torch.max(x_adv)
      loss=loss_fn(model(x_adv), y)
      
      
      #print(type(x_adv))
      '''
      g_t=mu*g_t.detach()+loss/abs(loss)
      g_t.backward()
      '''
      
      loss.backward()
      grad = x_adv.grad.detach()
      x_adv=x_adv+alpha*grad.sign()
      x_adv=torch.clip(x_adv,x-epsilon,x+epsilon)
      x_adv=x_adv.detach().clone()
    return x_adv



def mifgsm(model, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=20, decay=0.3):

    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    #x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
    g_t = loss_fn(model(x_adv), y) # calculate loss

    min_adv,max_adv=torch.min(x_adv),torch.max(x_adv)
    #loss.backward() # calculate gradient
    # fgsm: use gradient ascent on x_adv to maximize loss
    #grad = x_adv.grad.detach()
    #x_adv = x_adv + epsilon * grad.sign()

    # initialze momentum tensor
    momentum = torch.zeros_like(x).detach().to(device)
    mu=1
    g_t=0
    alpha=8/num_iter
    #epsilon=4/255
    ################ TODO: Strong baseline ####################
    for i in range(num_iter):
      #print(i)
      # TODO: Refer to the algorithm of MI-FGSM
      # Calculate the momentum and update
      #x_adv=DIM(x_adv)
      #print(type(x_adv))
      x_adv.requires_grad = True
      #min_adv,max_adv=torch.min(x_adv),torch.max(x_adv)
      loss=loss_fn(model(x_adv), y)
      
      
      #print(type(x_adv))
      '''
      g_t=mu*g_t.detach()+loss/abs(loss)
      g_t.backward()
      '''
      
      loss.backward()
      grad = x_adv.grad.detach()
      g_t=mu*g_t+grad/abs(grad)
      #print(grad.sign())
      #print(g_t.sign())
      x_adv=x_adv+alpha*g_t.sign()
      x_adv=torch.clip(x_adv,x-epsilon,x+epsilon)
      x_adv=x_adv.detach().clone()
    return x_adv

## Utils -- Attack
* Recall
  * ToTensor() can be seen as a function where $T(x) = x/255$.
  * Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.

* Inverse function
  * Inverse Normalize() can be seen as a function where $N^{-1}(x) = x*std+mean$ where $mean$ and $std$ are constants.
  * Inverse ToTensor() can be seen as a function where $T^{-1}(x) = x*255$.

* Special Noted
  * ToTensor() will also convert the image from shape (height, width, channel) to shape (channel, height, width), so we also need to transpose the shape back to original shape.
  * Since our dataloader samples a batch of data, what we need here is to transpose **(batch_size, channel, height, width)** back to **(batch_size, height, width, channel)** using np.transpose.

In [9]:
# perform adversarial attack and generate adversarial examples
def gen_adv_examples(model, loader, attack, loss_fn):
    model.eval()
    adv_names = []
    train_acc, train_loss = 0.0, 0.0
    for i, (x, y) in enumerate(loader):
        if i%5==0:
          print(i,"/",len(loader))
        x, y = x.to(device), y.to(device)
        x_adv = attack(model, x, y, loss_fn) # obtain adversarial examples
        yp = model(x_adv)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
        # store adversarial examples
        adv_ex = ((x_adv) * std + mean).clamp(0, 1) # to 0-1 scale
        adv_ex = (adv_ex * 255).clamp(0, 255) # 0-255 scale
        adv_ex = adv_ex.detach().cpu().data.numpy().round() # round to remove decimal part
        adv_ex = adv_ex.transpose((0, 2, 3, 1)) # transpose (bs, C, H, W) back to (bs, H, W, C)
        adv_examples = adv_ex if i == 0 else np.r_[adv_examples, adv_ex]
    return adv_examples, train_acc / len(loader.dataset), train_loss / len(loader.dataset)

# create directory which stores adversarial examples
def create_dir(data_dir, adv_dir, adv_examples, adv_names):
    if os.path.exists(adv_dir) is not True:
        _ = shutil.copytree(data_dir, adv_dir)
    for example, name in zip(adv_examples, adv_names):
        im = Image.fromarray(example.astype(np.uint8)) # image pixel value should be unsigned int
        im.save(os.path.join(adv_dir, name))

## Model / Loss Function

Model list is available [here](https://github.com/osmr/imgclsmob/blob/master/pytorch/pytorchcv/model_provider.py). Please select models which has _cifar10 suffix. Other kinds of models are prohibited, and it will be considered to be cheating if you use them. 

Note: Some of the models cannot be accessed/loaded. You can safely skip them since TA's model will not use those kinds of models.

In [10]:
# This function is used to check whether you use models pretrained on cifar10 instead of other datasets
def model_checker(model_name):
  assert ('cifar10' in model_name) and ('cifar100' not in model_name), 'The model selected is not pretrained on cifar10!'

In [11]:
from pytorchcv.model_provider import get_model as ptcv_get_model

model_name = 'resnet110_cifar10'
model_checker(model_name)

model = ptcv_get_model(model_name, pretrained=True).to(device)
loss_fn = nn.CrossEntropyLoss()

benign_acc, benign_loss = epoch_benign(model, adv_loader, loss_fn)
print(f'benign_acc = {benign_acc:.5f}, benign_loss = {benign_loss:.5f}')

benign_acc = 0.95000, benign_loss = 0.22690


In [12]:
################ BOSS BASELINE ######################

class ensembleNet(nn.Module):
    def __init__(self, model_names):
        super().__init__()
        
        
        
        
        self.models = nn.ModuleList([ptcv_get_model(name, pretrained=True) for name in model_names])
        self.model_names=model_names
    def forward(self, x):
        #################### TODO: boss baseline ###################
        ensemble_logits=torch.zeros(self.models[0](x).size()).to('cuda')
        for i, m in enumerate(self.models):
          #print(self.model_names[i])
          ensemble_logits+=m(x)
        ensemble_logits=ensemble_logits/len(self.models)
        # TODO: sum up logits from multiple models  
        return ensemble_logits
model_names = [
    'preresnet542bn_cifar10',
    'pyramidnet272_a200_bn_cifar10',
    'preresnet1001_cifar10',
    'wrn16_10_cifar10',
    'pyramidnet110_a84_cifar10',
    'nin_cifar10',
    'resnet110_cifar10',
    'sepreresnet542bn_cifar10',
    'sepreresnet272bn_cifar10',
    'preresnet56_cifar10',
    'resnext29_16x64d_cifar10',
]
model_names=['wrn16_10_cifar10', 
             'pyramidnet272_a200_bn_cifar10', 
             'preresnet1001_cifar10', 
             'wrn16_10_cifar10', 
             'pyramidnet110_a84_cifar10', 
             'nin_cifar10', 
             'resnet110_cifar10', 
             'preresnet56_cifar10', 
             'resnext29_16x64d_cifar10']
model_names=['shakeshakeresnet26_2x32d_cifar10',
             'preresnet272bn_cifar10',
             'seresnet56_cifar10',
             'sepreresnet56_cifar10',
             'ror3_110_cifar10',
             'preresnet110_cifar10',
             'resnet164bn_cifar10',
             'resnet110_cifar10',
             'seresnet110_cifar10',
             'seresnet164bn_cifar10',
             'sepreresnet164bn_cifar10',
             'diaresnet164bn_cifar10',
             'ror3_164_cifar10',
             
             ]
model_dict={'preresnet20_cifar10':[41.27,6.51],
'resnet20_cifar10':[41.29,5.97],
'seresnet20_cifar10':[41.34,6.01],
'sepreresnet20_cifar10':[41.35,6.18],
'diapreresnet20_cifar10':[41.52,6.42],
'diaresnet20_cifar10':[41.54,6.22],
'densenet40_k12_cifar10':[74.89,6.43],
'shakeshakeresnet20_2x16d_cifar10':[81.78,5.15],
'ror3_56_cifar10':[113.43,5.43],
'preresnet56_cifar10':[127.03,4.49],
'resnet56_cifar10':[127.06,4.52],
'seresnet56_cifar10':[127.19,4.13],
'sepreresnet56_cifar10':[127.2,4.51],
'diapreresnet56_cifar10':[129.28,4.83],
'diaresnet56_cifar10':[129.31,5.05],
'densenet100_k12_bc_cifar10':[210.8,5.61],
'nin_cifar10':[222.97,7.43],
'ror3_110_cifar10':[242.07,4.35],
'preresnet164bn_cifar10':[255.08,3.64],
'resnet164bn_cifar10':[255.31,3.68],
'preresnet110_cifar10':[255.68,3.86],
'resnet110_cifar10':[255.7,3.69],
'sepreresnet110_cifar10':[255.98,4.54],
'seresnet110_cifar10':[255.98,3.63],
'sepreresnet164bn_cifar10':[256.32,3.73],
'seresnet164bn_cifar10':[256.55,3.39],
'diapreresnet110_cifar10':[264.69,4.25],
'diaresnet110_cifar10':[264.71,4.1],
'densenet40_k24_bc_cifar10':[293.09,4.52],
'xdensenet40_2_k24_bc_cifar10':[293.09,5.31],
'densenet190_k40_bc_cifar10':[298.45,4.16],
'diapreresnet164bn_cifar10':[343.37,3.56],
'diaresnet164bn_cifar10':[343.6,3.5],
'ror3_164_cifar10':[370.72,3.93],
'pyramidnet110_a48_cifar10':[408.37,3.72],
'preresnet272bn_cifar10':[420.38,3.25],
'resnet272bn_cifar10':[420.61,3.33],
'sepreresnet272bn_cifar10':[422.45,3.39],
'seresnet272bn_cifar10':[422.68,3.39],
'shakeshakeresnet26_2x32d_cifar10':[428.89,3.17],
'densenet40_k36_bc_cifar10':[654.6,4.04],
'xdensenet40_2_k36_bc_cifar10':[654.6,4.37],
'pyramidnet110_a84_cifar10':[778.15,2.98],
'resnext29_32x4d_cifar10':[780.55,3.15],
'preresnet542bn_cifar10':[833.64,3.14],
'resnet542bn_cifar10':[833.87,3.43],
'sepreresnet542bn_cifar10':[837.78,3.08],
'seresnet542bn_cifar10':[838.01,3.47],
'rir_cifar10':[1281.08,3.28],
'densenet100_k12_cifar10':[1353.55,3.66],
'preresnet1001_cifar10':[1536.18,2.65],
'resnet1001_cifar10':[1536.4,3.28],
'wrn16_10_cifar10':[2414.04,2.93],
'preresnet1202_cifar10':[2857.14,3.39],
'resnet1202_cifar10':[2857.17,3.53],
'wrn20_10_1bit_cifar10':[4019.14,3.26],
'wrn20_10_32bit_cifar10':[4019.14,3.14],
'pyramidnet272_a200_bn_cifar10':[4541.36,2.39],
'pyramidnet200_a240_bn_cifar10':[4563.4,2.44],
'pyramidnet164_a270_bn_cifar10':[4608.81,2.42],
'pyramidnet236_a220_bn_cifar10':[4631.32,2.47],
'pyramidnet110_a270_cifar10':[4730.6,2.51],
'resnext272_2x32d_cifar10':[4867.11,2.74],
'wrn40_8_cifar10':[5176.9,2.37],
'wrn28_10_cifar10':[5246.98,2.39],
'densenet100_k24_cifar10':[5354.19,3.13],
'densenet40_k12_bc_cifar10':[5519.54,2.67],
'resnext272_1x64d_cifar10':[6565.15,2.55],
'densenet250_k24_bc_cifar10':[9400.45,2.52],
'resnext29_16x64d_cifar10':[10709.34,2.41],}
import random
model_names=[]
total_memory=0
total_error=0

model_list=list(model_dict.keys())
'''
random.shuffle(model_list)
for m in model_list:

    if model_dict[m][0]>1000 and len(model_names)<10:
        print(model_dict[m][0])
        continue
    total_memory+=model_dict[m][0]
    
    if total_memory<=9000:
        model_names.append(m)
        total_error+=model_dict[m][1]
    else:
        total_memory-=model_dict[m][0]
        break
'''
max_len=25
for i,m in enumerate(model_list):
    if i%2==0 and model_dict[m][1]<1000 and len(model_names)<max_len:
        model_names.append(m)
        total_error+=model_dict[m][1]
        total_memory+=model_dict[m][0]
best_model1=model_names
'''
model_names=['seresnet164bn_cifar10',
'seresnet110_cifar10',
'preresnet164bn_cifar10',
'shakeshakeresnet26_2x32d_cifar10',
'resnet164bn_cifar10',
'resnet110_cifar10',
'preresnet272bn_cifar10',
'sepreresnet164bn_cifar10',
'seresnet56_cifar10',
'diaresnet164bn_cifar10',
'resnet272bn_cifar10',
'diapreresnet164bn_cifar10',
'preresnet110_cifar10',
'sepreresnet272bn_cifar10',
'seresnet272bn_cifar10',
'preresnet56_cifar10',
'sepreresnet56_cifar10',
'resnet56_cifar10',
'diaresnet110_cifar10',
'pyramidnet110_a48_cifar10',
'ror3_164_cifar10',
'diapreresnet110_cifar10',
'densenet190_k40_bc_cifar10',
'ror3_110_cifar10',
'diapreresnet56_cifar10',
'pyramidnet110_a84_cifar10',
'sepreresnet110_cifar10',
'shakeshakeresnet20_2x16d_cifar10',
'resnext29_32x4d_cifar10',]
'''
#model_names=random.choices(model_names,k=20)
print(model_names)
print(len(model_names))
print("total memory:",total_memory)
print("average error:",total_error/len(model_names))



['preresnet20_cifar10', 'seresnet20_cifar10', 'diapreresnet20_cifar10', 'densenet40_k12_cifar10', 'ror3_56_cifar10', 'resnet56_cifar10', 'sepreresnet56_cifar10', 'diaresnet56_cifar10', 'nin_cifar10', 'preresnet164bn_cifar10', 'preresnet110_cifar10', 'sepreresnet110_cifar10', 'sepreresnet164bn_cifar10', 'diapreresnet110_cifar10', 'densenet40_k24_bc_cifar10', 'densenet190_k40_bc_cifar10', 'diaresnet164bn_cifar10', 'pyramidnet110_a48_cifar10', 'resnet272bn_cifar10', 'seresnet272bn_cifar10', 'densenet40_k36_bc_cifar10', 'pyramidnet110_a84_cifar10', 'preresnet542bn_cifar10', 'sepreresnet542bn_cifar10', 'rir_cifar10']
25
total memory: 8778.79
average error: 4.4588


In [13]:
total_memory=0
total_error=0
resnet_family=['densenet100_k12_cifar10',
#'densenet100_k24_cifar10',
'densenet100_k12_bc_cifar10',
'densenet190_k40_bc_cifar10',
'densenet250_k24_bc_cifar10',
'densenet40_k12_bc_cifar10',
'densenet40_k12_cifar10',
'densenet40_k24_bc_cifar10',
'densenet40_k36_bc_cifar10',
'diapreresnet110_cifar10',
'diapreresnet164bn_cifar10',
'diapreresnet20_cifar10',
'diapreresnet56_cifar10',
'diaresnet110_cifar10',
'diaresnet164bn_cifar10',
'diaresnet20_cifar10',
'diaresnet56_cifar10',
'resnet272bn_cifar10',
'resnet56_cifar10',
'rir_cifar10',
'ror3_164_cifar10',
'sepreresnet110_cifar10',
'sepreresnet164bn_cifar10',
'sepreresnet20_cifar10',
'sepreresnet272bn_cifar10',
'sepreresnet542bn_cifar10',
'sepreresnet56_cifar10',
'seresnet110_cifar10',
'seresnet164bn_cifar10',
'seresnet20_cifar10',
'seresnet272bn_cifar10',
'seresnet542bn_cifar10',
'seresnet56_cifar10',]
#['', 'sepreresnet20_cifar10', '', '', 'sepreresnet56_cifar10', 'densenet100_k12_bc_cifar10', 'preresnet164bn_cifar10', '', 'sepreresnet164bn_cifar10', 'diaresnet110_cifar10', 'densenet190_k40_bc_cifar10', 'ror3_164_cifar10', 'resnet272bn_cifar10', 'shakeshakeresnet26_2x32d_cifar10', 'pyramidnet110_a84_cifar10', '', 'rir_cifar10', 'resnet1001
model_index=[]
for i,m in enumerate(model_list):
    if m in model_names:
    #if i<33:
        total_memory+=model_dict[m][0]
        total_error+=model_dict[m][1]
        model_index.append(i)
        
print(len(model_names))
print(model_names)
print(total_memory)
print(total_error/len(model_names))

25
['preresnet20_cifar10', 'seresnet20_cifar10', 'diapreresnet20_cifar10', 'densenet40_k12_cifar10', 'ror3_56_cifar10', 'resnet56_cifar10', 'sepreresnet56_cifar10', 'diaresnet56_cifar10', 'nin_cifar10', 'preresnet164bn_cifar10', 'preresnet110_cifar10', 'sepreresnet110_cifar10', 'sepreresnet164bn_cifar10', 'diapreresnet110_cifar10', 'densenet40_k24_bc_cifar10', 'densenet190_k40_bc_cifar10', 'diaresnet164bn_cifar10', 'pyramidnet110_a48_cifar10', 'resnet272bn_cifar10', 'seresnet272bn_cifar10', 'densenet40_k36_bc_cifar10', 'pyramidnet110_a84_cifar10', 'preresnet542bn_cifar10', 'sepreresnet542bn_cifar10', 'rir_cifar10']
8778.79
4.4588


In [14]:
mode="error"
#####sort by loss######
if mode=="loss":
    model_dict={'preresnet20_cifar10': [41.27, 14.002521896362305], 'resnet20_cifar10': [41.29, 15.455768013000489], 'seresnet20_cifar10': [41.34, 12.153068504333497], 'sepreresnet20_cifar10': [41.35, 12.329749183654785], 'diapreresnet20_cifar10': [41.52, 11.258948059082032], 'diaresnet20_cifar10': [41.54, 9.293731651306153], 'densenet40_k12_cifar10': [74.89, 12.705183525085449], 'shakeshakeresnet20_2x16d_cifar10': [81.78, 10.11400177001953], 'ror3_56_cifar10': [113.43, 9.83536756515503], 'preresnet56_cifar10': [127.03, 11.35746124267578], 'resnet56_cifar10': [127.06, 11.712560501098633], 'seresnet56_cifar10': [127.19, 9.609673614501952], 'sepreresnet56_cifar10': [127.2, 10.337662391662597], 'diapreresnet56_cifar10': [129.28, 10.798598403930663], 'diaresnet56_cifar10': [129.31, 9.487401542663575], 'densenet100_k12_bc_cifar10': [210.8, 10.517892322540284], 'nin_cifar10': [222.97, 9.284525337219238], 'ror3_110_cifar10': [242.07, 8.537963504791259], 'preresnet164bn_cifar10': [255.08, 11.852857666015625], 'resnet164bn_cifar10': [255.31, 9.105857219696045], 'preresnet110_cifar10': [255.68, 11.42756929397583], 'resnet110_cifar10': [255.7, 11.505998649597167], 'sepreresnet110_cifar10': [255.98, 12.038713569641112], 'seresnet110_cifar10': [255.98, 10.485417442321777], 'sepreresnet164bn_cifar10': [256.32, 11.334516639709472], 'seresnet164bn_cifar10': [256.55, 9.558498840332032], 'diapreresnet110_cifar10': [264.69, 12.079894485473632], 'diaresnet110_cifar10': [264.71, 11.287718925476074], 'densenet40_k24_bc_cifar10': [293.09, 12.832738609313965], 'xdensenet40_2_k24_bc_cifar10': [293.09, 13.962934150695801], 'densenet190_k40_bc_cifar10': [298.45, 11.011504554748536], 'diapreresnet164bn_cifar10': [343.37, 9.987324905395507], 'diaresnet164bn_cifar10': [343.6, 11.863894233703613], 'ror3_164_cifar10': [370.72, 10.107543544769287], 'pyramidnet110_a48_cifar10': [408.37, 13.034896545410156], 'preresnet272bn_cifar10': [420.38, 10.406990242004394], 'resnet272bn_cifar10': [420.61, 12.495949745178223], 'sepreresnet272bn_cifar10': [422.45, 12.36975715637207], 'seresnet272bn_cifar10': [422.68, 8.959838218688965], 'shakeshakeresnet26_2x32d_cifar10': [428.89, 10.126092185974121], 'densenet40_k36_bc_cifar10': [654.6, 10.228862380981445], 'xdensenet40_2_k36_bc_cifar10': [654.6, 9.984689674377442], 'pyramidnet110_a84_cifar10': [778.15, 14.02359088897705], 'resnext29_32x4d_cifar10': [780.55, 17.76765525817871], 'preresnet542bn_cifar10': [833.64, 9.7698171043396], 'resnet542bn_cifar10': [833.87, 9.412583045959472], 'sepreresnet542bn_cifar10': [837.78, 10.032699031829834], 'seresnet542bn_cifar10': [838.01, 10.542131252288819], 'rir_cifar10': [1281.08, 12.140970497131347], 'densenet100_k12_cifar10': [1353.55, 11.513689994812012], 'preresnet1001_cifar10': [1536.18, 9.667638549804687], 'resnet1001_cifar10': [1536.4, 10.535249824523925], 'wrn16_10_cifar10': [2414.04, 15.17474723815918], 'preresnet1202_cifar10': [2857.14, 9.817216911315917], 'resnet1202_cifar10': [2857.17, 8.871091079711913], 'wrn20_10_1bit_cifar10': [4019.14, 15.239664878845215], 'wrn20_10_32bit_cifar10': [4019.14, 13.293598175048828], 'pyramidnet272_a200_bn_cifar10': [4541.36, 11.890794334411622], 'pyramidnet200_a240_bn_cifar10': [4563.4, 11.384682960510254], 'pyramidnet164_a270_bn_cifar10': [4608.81, 11.562754898071288], 'pyramidnet236_a220_bn_cifar10': [4631.32, 12.45128273010254], 'pyramidnet110_a270_cifar10': [4730.6, 18.649833183288575], 'resnext272_2x32d_cifar10': [4867.11, 11.92292751312256], 'wrn40_8_cifar10': [5176.9, 12.256089820861817], 'wrn28_10_cifar10': [5246.98, 16.313759994506835], 'densenet100_k24_cifar10': [5354.19, 12.105612030029297], 'densenet40_k12_bc_cifar10': [5519.54, 16.207676773071288], 'resnext272_1x64d_cifar10': [6565.15, 11.917600746154784], 'densenet250_k24_bc_cifar10': [9400.45, 12.131968269348144], 'resnext29_16x64d_cifar10': [10709.34, 10.799761257171632]}
elif mode=="acc":
    model_dict={'preresnet20_cifar10': [41.27, 0.0], 'resnet20_cifar10': [41.29, 0.0], 'seresnet20_cifar10': [41.34, 0.01], 'sepreresnet20_cifar10': [41.35, 0.02], 'diapreresnet20_cifar10': [41.52, 0.02], 'diaresnet20_cifar10': [41.54, 0.005], 'densenet40_k12_cifar10': [74.89, 0.0], 'shakeshakeresnet20_2x16d_cifar10': [81.78, 0.01], 'ror3_56_cifar10': [113.43, 0.0], 'preresnet56_cifar10': [127.03, 0.015], 'resnet56_cifar10': [127.06, 0.01], 'seresnet56_cifar10': [127.19, 0.01], 'sepreresnet56_cifar10': [127.2, 0.015], 'diapreresnet56_cifar10': [129.28, 0.06], 'diaresnet56_cifar10': [129.31, 0.02], 'densenet100_k12_bc_cifar10': [210.8, 0.005], 'nin_cifar10': [222.97, 0.0], 'ror3_110_cifar10': [242.07, 0.02], 'preresnet164bn_cifar10': [255.08, 0.03], 'resnet164bn_cifar10': [255.31, 0.0], 'preresnet110_cifar10': [255.68, 0.045], 'resnet110_cifar10': [255.7, 0.01], 'sepreresnet110_cifar10': [255.98, 0.0], 'seresnet110_cifar10': [255.98, 0.025], 'sepreresnet164bn_cifar10': [256.32, 0.075], 'seresnet164bn_cifar10': [256.55, 0.01], 'diapreresnet110_cifar10': [264.69, 0.065], 'diaresnet110_cifar10': [264.71, 0.01], 'densenet40_k24_bc_cifar10': [293.09, 0.01], 'xdensenet40_2_k24_bc_cifar10': [293.09, 0.01], 'densenet190_k40_bc_cifar10': [298.45, 0.02], 'diapreresnet164bn_cifar10': [343.37, 0.105], 'diaresnet164bn_cifar10': [343.6, 0.035], 'ror3_164_cifar10': [370.72, 0.045], 'pyramidnet110_a48_cifar10': [408.37, 0.025], 'preresnet272bn_cifar10': [420.38, 0.075], 'resnet272bn_cifar10': [420.61, 0.015], 'sepreresnet272bn_cifar10': [422.45, 0.045], 'seresnet272bn_cifar10': [422.68, 0.105], 'shakeshakeresnet26_2x32d_cifar10': [428.89, 0.0], 'densenet40_k36_bc_cifar10': [654.6, 0.0], 'xdensenet40_2_k36_bc_cifar10': [654.6, 0.0], 'pyramidnet110_a84_cifar10': [778.15, 0.04], 'resnext29_32x4d_cifar10': [780.55, 0.0], 'preresnet542bn_cifar10': [833.64, 0.145], 'resnet542bn_cifar10': [833.87, 0.12], 'sepreresnet542bn_cifar10': [837.78, 0.14], 'seresnet542bn_cifar10': [838.01, 0.08], 'rir_cifar10': [1281.08, 0.015], 'densenet100_k12_cifar10': [1353.55, 0.025], 'preresnet1001_cifar10': [1536.18, 0.105], 'resnet1001_cifar10': [1536.4, 0.09], 'wrn16_10_cifar10': [2414.04, 0.005], 'preresnet1202_cifar10': [2857.14, 0.085], 'resnet1202_cifar10': [2857.17, 0.14], 'wrn20_10_1bit_cifar10': [4019.14, 0.005], 'wrn20_10_32bit_cifar10': [4019.14, 0.0], 'pyramidnet272_a200_bn_cifar10': [4541.36, 0.005], 'pyramidnet200_a240_bn_cifar10': [4563.4, 0.015], 'pyramidnet164_a270_bn_cifar10': [4608.81, 0.005], 'pyramidnet236_a220_bn_cifar10': [4631.32, 0.005], 'pyramidnet110_a270_cifar10': [4730.6, 0.045], 'resnext272_2x32d_cifar10': [4867.11, 0.07], 'wrn40_8_cifar10': [5176.9, 0.035], 'wrn28_10_cifar10': [5246.98, 0.005], 'densenet100_k24_cifar10': [5354.19, 0.05], 'densenet40_k12_bc_cifar10': [5519.54, 0.0], 'resnext272_1x64d_cifar10': [6565.15, 0.095], 'densenet250_k24_bc_cifar10': [9400.45, 0.12], 'resnext29_16x64d_cifar10': [10709.34, 0.0]}
elif mode=="error":
    model_dict={'preresnet20_cifar10': [41.27, 6.51], 'resnet20_cifar10': [41.29, 5.97], 'seresnet20_cifar10': [41.34, 6.01], 'sepreresnet20_cifar10': [41.35, 6.18], 'diapreresnet20_cifar10': [41.52, 6.42], 'diaresnet20_cifar10': [41.54, 6.22], 'densenet40_k12_cifar10': [74.89, 6.43], 'shakeshakeresnet20_2x16d_cifar10': [81.78, 5.15], 'ror3_56_cifar10': [113.43, 5.43], 'preresnet56_cifar10': [127.03, 4.49], 'resnet56_cifar10': [127.06, 4.52], 'seresnet56_cifar10': [127.19, 4.13], 'sepreresnet56_cifar10': [127.2, 4.51], 'diapreresnet56_cifar10': [129.28, 4.83], 'diaresnet56_cifar10': [129.31, 5.05], 'densenet100_k12_bc_cifar10': [210.8, 5.61], 'nin_cifar10': [222.97, 7.43], 'ror3_110_cifar10': [242.07, 4.35], 'preresnet164bn_cifar10': [255.08, 3.64], 'resnet164bn_cifar10': [255.31, 3.68], 'preresnet110_cifar10': [255.68, 3.86], 'resnet110_cifar10': [255.7, 3.69], 'sepreresnet110_cifar10': [255.98, 4.54], 'seresnet110_cifar10': [255.98, 3.63], 'sepreresnet164bn_cifar10': [256.32, 3.73], 'seresnet164bn_cifar10': [256.55, 3.39], 'diapreresnet110_cifar10': [264.69, 4.25], 'diaresnet110_cifar10': [264.71, 4.1], 'densenet40_k24_bc_cifar10': [293.09, 4.52], 'xdensenet40_2_k24_bc_cifar10': [293.09, 5.31], 'densenet190_k40_bc_cifar10': [298.45, 4.16], 'diapreresnet164bn_cifar10': [343.37, 3.56], 'diaresnet164bn_cifar10': [343.6, 3.5], 'ror3_164_cifar10': [370.72, 3.93], 'pyramidnet110_a48_cifar10': [408.37, 3.72], 'preresnet272bn_cifar10': [420.38, 3.25], 'resnet272bn_cifar10': [420.61, 3.33], 'sepreresnet272bn_cifar10': [422.45, 3.39], 'seresnet272bn_cifar10': [422.68, 3.39], 'shakeshakeresnet26_2x32d_cifar10': [428.89, 3.17], 'densenet40_k36_bc_cifar10': [654.6, 4.04], 'xdensenet40_2_k36_bc_cifar10': [654.6, 4.37], 'pyramidnet110_a84_cifar10': [778.15, 2.98], 'resnext29_32x4d_cifar10': [780.55, 3.15], 'preresnet542bn_cifar10': [833.64, 3.14], 'resnet542bn_cifar10': [833.87, 3.43], 'sepreresnet542bn_cifar10': [837.78, 3.08], 'seresnet542bn_cifar10': [838.01, 3.47], 'rir_cifar10': [1281.08, 3.28], 'densenet100_k12_cifar10': [1353.55, 3.66], 'preresnet1001_cifar10': [1536.18, 2.65], 'resnet1001_cifar10': [1536.4, 3.28], 'wrn16_10_cifar10': [2414.04, 2.93], 'preresnet1202_cifar10': [2857.14, 3.39], 'resnet1202_cifar10': [2857.17, 3.53], 'wrn20_10_1bit_cifar10': [4019.14, 3.26], 'wrn20_10_32bit_cifar10': [4019.14, 3.14], 'pyramidnet272_a200_bn_cifar10': [4541.36, 2.39], 'pyramidnet200_a240_bn_cifar10': [4563.4, 2.44], 'pyramidnet164_a270_bn_cifar10': [4608.81, 2.42], 'pyramidnet236_a220_bn_cifar10': [4631.32, 2.47], 'pyramidnet110_a270_cifar10': [4730.6, 2.51], 'resnext272_2x32d_cifar10': [4867.11, 2.74], 'wrn40_8_cifar10': [5176.9, 2.37], 'wrn28_10_cifar10': [5246.98, 2.39], 'densenet100_k24_cifar10': [5354.19, 3.13], 'densenet40_k12_bc_cifar10': [5519.54, 2.67], 'resnext272_1x64d_cifar10': [6565.15, 2.55], 'densenet250_k24_bc_cifar10': [9400.45, 2.52], 'resnext29_16x64d_cifar10': [10709.34, 2.41]}



In [15]:
#print(list(model_dict.items())[0][0])
model_list=list(model_dict.items())
#print(model_list[1][1][1])
model_list=sorted(model_list,key=lambda x:x[1][1],reverse=False)
print(model_list)
model_list_less_1000=[]
model_list_high_cp=[]
model_names=[]
total_memory=0
for m in model_list:
    
    if m[1][0]>1500:continue
    total_memory+=m[1][0]
    #print(total_memory)
    #print(total_memory)
    if m[1][0]<1500 and total_memory<14000:
        model_list_high_cp.append(m[0])
    if m[1][0]<1000 and len(model_list_less_1000)<45:
        model_list_less_1000.append(m[0])
    if i%2==0 or m[1][0]>1000 or len(model_names)>33:
        continue
    model_names.append(m[0])
best_model2=model_names
#print(model_names)
#print(model_dict)
max_slice=23
model_names=model_names[1:max_slice]
for m in resnet_family:
    if len(model_names)>=30:
        break
    if m not in model_names:
        model_names.append(m)
#print(model_names)
##################test ensemble model###################
#model_names=resnet_family
def calculator(model_names,model_list):
    global model_dict
    total_memory=0
    total_error=0
    model_index=[]
    for i,m in enumerate(model_list):
        
        m=m[0]
        #print("mmm:",m,model_names)
        if m in model_names:
            #print("Test")
            #print(model_dict[m])
        #if i<33:
            #print(model_dict[m][0])
            total_memory+=model_dict[m][0]
            total_error+=model_dict[m][1]
            model_index.append(i)
    return total_memory,total_error
best_model2=sorted(best_model2)
for m in best_model2:
    print(m)
#print(model_names)
#print(len(model_names))
#print(total_memory)
#print(total_error/len(model_names))

[('wrn40_8_cifar10', [5176.9, 2.37]), ('pyramidnet272_a200_bn_cifar10', [4541.36, 2.39]), ('wrn28_10_cifar10', [5246.98, 2.39]), ('resnext29_16x64d_cifar10', [10709.34, 2.41]), ('pyramidnet164_a270_bn_cifar10', [4608.81, 2.42]), ('pyramidnet200_a240_bn_cifar10', [4563.4, 2.44]), ('pyramidnet236_a220_bn_cifar10', [4631.32, 2.47]), ('pyramidnet110_a270_cifar10', [4730.6, 2.51]), ('densenet250_k24_bc_cifar10', [9400.45, 2.52]), ('resnext272_1x64d_cifar10', [6565.15, 2.55]), ('preresnet1001_cifar10', [1536.18, 2.65]), ('densenet40_k12_bc_cifar10', [5519.54, 2.67]), ('resnext272_2x32d_cifar10', [4867.11, 2.74]), ('wrn16_10_cifar10', [2414.04, 2.93]), ('pyramidnet110_a84_cifar10', [778.15, 2.98]), ('sepreresnet542bn_cifar10', [837.78, 3.08]), ('densenet100_k24_cifar10', [5354.19, 3.13]), ('preresnet542bn_cifar10', [833.64, 3.14]), ('wrn20_10_32bit_cifar10', [4019.14, 3.14]), ('resnext29_32x4d_cifar10', [780.55, 3.15]), ('shakeshakeresnet26_2x32d_cifar10', [428.89, 3.17]), ('preresnet272bn_ci

In [16]:
#model_list_high_cp=sorted(model_list_high_cp)
print(len(model_list_high_cp))
for m in model_list_high_cp:
    print(m)

27
pyramidnet110_a84_cifar10
sepreresnet542bn_cifar10
preresnet542bn_cifar10
resnext29_32x4d_cifar10
shakeshakeresnet26_2x32d_cifar10
preresnet272bn_cifar10
rir_cifar10
resnet272bn_cifar10
seresnet164bn_cifar10
sepreresnet272bn_cifar10
seresnet272bn_cifar10
resnet542bn_cifar10
seresnet542bn_cifar10
diaresnet164bn_cifar10
diapreresnet164bn_cifar10
seresnet110_cifar10
preresnet164bn_cifar10
densenet100_k12_cifar10
resnet164bn_cifar10
resnet110_cifar10
pyramidnet110_a48_cifar10
sepreresnet164bn_cifar10
preresnet110_cifar10
ror3_164_cifar10
densenet40_k36_bc_cifar10
diaresnet110_cifar10
seresnet56_cifar10


## iteration get the fgsm loss

In [17]:
'''
from pytorchcv.model_provider import get_model as ptcv_get_model
for m in model_list:
    model_name = m
    model_checker(model_name)

    model = ptcv_get_model(model_name, pretrained=True).to(device)
    loss_fn = nn.CrossEntropyLoss()
    benign_acc, benign_loss = epoch_benign(model, adv_loader, loss_fn)
    adv_examples, fgsm_acc, fgsm_loss = gen_adv_examples(model, adv_loader, mifgsm, loss_fn)
    model_dict[m][1]=fgsm_acc
    #print(m,adv_examples)
'''

'\nfrom pytorchcv.model_provider import get_model as ptcv_get_model\nfor m in model_list:\n    model_name = m\n    model_checker(model_name)\n\n    model = ptcv_get_model(model_name, pretrained=True).to(device)\n    loss_fn = nn.CrossEntropyLoss()\n    benign_acc, benign_loss = epoch_benign(model, adv_loader, loss_fn)\n    adv_examples, fgsm_acc, fgsm_loss = gen_adv_examples(model, adv_loader, mifgsm, loss_fn)\n    model_dict[m][1]=fgsm_acc\n    #print(m,adv_examples)\n'

In [18]:
import copy

print(len(best_model1))
print(best_model1)
print(len(best_model2))
print(best_model2)
merge_model=copy.deepcopy(best_model2)
for i,m in enumerate(best_model1):
    if m  not in merge_model and len(merge_model)<50:
        merge_model.append(best_model2[i])
print(len(merge_model))
print(merge_model)
        
        
        
        

25
['preresnet20_cifar10', 'seresnet20_cifar10', 'diapreresnet20_cifar10', 'densenet40_k12_cifar10', 'ror3_56_cifar10', 'resnet56_cifar10', 'sepreresnet56_cifar10', 'diaresnet56_cifar10', 'nin_cifar10', 'preresnet164bn_cifar10', 'preresnet110_cifar10', 'sepreresnet110_cifar10', 'sepreresnet164bn_cifar10', 'diapreresnet110_cifar10', 'densenet40_k24_bc_cifar10', 'densenet190_k40_bc_cifar10', 'diaresnet164bn_cifar10', 'pyramidnet110_a48_cifar10', 'resnet272bn_cifar10', 'seresnet272bn_cifar10', 'densenet40_k36_bc_cifar10', 'pyramidnet110_a84_cifar10', 'preresnet542bn_cifar10', 'sepreresnet542bn_cifar10', 'rir_cifar10']
34
['densenet190_k40_bc_cifar10', 'densenet40_k24_bc_cifar10', 'densenet40_k36_bc_cifar10', 'diapreresnet110_cifar10', 'diapreresnet164bn_cifar10', 'diaresnet110_cifar10', 'diaresnet164bn_cifar10', 'preresnet110_cifar10', 'preresnet164bn_cifar10', 'preresnet272bn_cifar10', 'preresnet542bn_cifar10', 'preresnet56_cifar10', 'pyramidnet110_a48_cifar10', 'pyramidnet110_a84_cifar1

## FGSM

In [19]:
'''
from datetime import datetime
t=str(datetime.now())
f=open('log/'+t+'.txt','a+')
best_loss=0
best_index=[]
best_name=''
epoch=0
fgsm_acc=0
fgsm_loss=0
print(f'mifgsm_acc = {fgsm_acc:.5f}, fgsm_loss = {fgsm_loss:.5f}',model_index,model_names,file=f)
print(f'mifgsm_acc = {fgsm_acc:.5f}, fgsm_loss = {fgsm_loss:.5f}',model_index,model_names)
while True:

    
    model_names = random.choices(model_list,k=25)
    temp=[]
    for m in model_names:
        temp.append(m[0])
    model_names=temp
    #print(model_names)

    mem,loss=calculator(model_names,model_list)

    if mem>7500:continue
    if epoch==60:break
    try:
        
        model = ensembleNet(model_names).to(device)
        loss_fn = nn.CrossEntropyLoss()

        benign_acc, benign_loss = epoch_benign(model, adv_loader, loss_fn)

        #print("Start")
        adv_examples, fgsm_acc, fgsm_loss = gen_adv_examples(model, adv_loader, mifgsm, loss_fn)
        if fgsm_loss>best_loss:
            best_loss=fgsm_loss
            best_index=model_index
            best_name=model_names
        
    except:
        #print("end")
        continue
    #print(f'mifgsm_acc = {fgsm_acc:.5f}, fgsm_loss = {fgsm_loss:.5f}'+model_index+model_names)
        
    print("epoch:",epoch)
    epoch+=1
    print(f'mifgsm_acc = {fgsm_acc:.5f}, fgsm_loss = {fgsm_loss:.5f}',model_index,model_names,file=f)
    print(f'mifgsm_acc = {fgsm_acc:.5f}, fgsm_loss = {fgsm_loss:.5f}',model_index,model_names)
    print(mem,loss)
print(best_loss)
print(best_index)
print(best_name)
'''
'''
create_dir(root, 'mifgsm', adv_examples, adv_names)
print(len(model_names))
print("mmode:",mode)
print("max slice:",max_slice)
print("model index:",model_index)
print("total memory:",total_memory)
print("average loss:",total_error/len(model_names))
print("alpha",alpha)
'''


'\ncreate_dir(root, \'mifgsm\', adv_examples, adv_names)\nprint(len(model_names))\nprint("mmode:",mode)\nprint("max slice:",max_slice)\nprint("model index:",model_index)\nprint("total memory:",total_memory)\nprint("average loss:",total_error/len(model_names))\nprint("alpha",alpha)\n'

In [20]:
#print(best_loss)
#print(best_index)
#print(best_name)
#best_name=sorted(best_name)
#mem,loss=calculator(best_name,model_list)
#print(mem,loss)

In [21]:
#model_names=random.choices(merge_model,k=23)
#print(len(model_names))
print(calculator(model_list_high_cp,model_list))

(13954.82, 95.32000000000002)


In [22]:
print(model_list_high_cp)

['pyramidnet110_a84_cifar10', 'sepreresnet542bn_cifar10', 'preresnet542bn_cifar10', 'resnext29_32x4d_cifar10', 'shakeshakeresnet26_2x32d_cifar10', 'preresnet272bn_cifar10', 'rir_cifar10', 'resnet272bn_cifar10', 'seresnet164bn_cifar10', 'sepreresnet272bn_cifar10', 'seresnet272bn_cifar10', 'resnet542bn_cifar10', 'seresnet542bn_cifar10', 'diaresnet164bn_cifar10', 'diapreresnet164bn_cifar10', 'seresnet110_cifar10', 'preresnet164bn_cifar10', 'densenet100_k12_cifar10', 'resnet164bn_cifar10', 'resnet110_cifar10', 'pyramidnet110_a48_cifar10', 'sepreresnet164bn_cifar10', 'preresnet110_cifar10', 'ror3_164_cifar10', 'densenet40_k36_bc_cifar10', 'diaresnet110_cifar10', 'seresnet56_cifar10']


In [23]:

#model_names = best_name

model_names=model_list_high_cp
#model_names=random.choices(resnet_family,k=27)
print(len(model_names))
model = ensembleNet(model_names).to(device)
loss_fn = nn.CrossEntropyLoss()

benign_acc, benign_loss = epoch_benign(model, adv_loader, loss_fn)


adv_examples, fgsm_acc, fgsm_loss = gen_adv_examples(model, adv_loader, mifgsm, loss_fn)
print(f'mifgsm_acc = {fgsm_acc:.5f}, fgsm_loss = {fgsm_loss:.5f}')



create_dir(root, 'mifgsm', adv_examples, adv_names)
print(len(model_names))
print("mmode:",mode)
print("max slice:",max_slice)
print("model index:",model_index)
print("total memory:",total_memory)
print("average loss:",total_error/len(model_names))
print("alpha",alpha)
%cd mifgsm
!tar zcvf ../mifgsm.tgz *
%cd ..


27


KeyboardInterrupt: 

In [ ]:
model_names=sorted(model_names)
for m in model_names:
    print(m)

('densenet100_k12_cifar10', [1353.55, 3.66])
('densenet40_k36_bc_cifar10', [654.6, 4.04])
('diapreresnet164bn_cifar10', [343.37, 3.56])
('diaresnet110_cifar10', [264.71, 4.1])
('diaresnet164bn_cifar10', [343.6, 3.5])
('preresnet110_cifar10', [255.68, 3.86])
('preresnet164bn_cifar10', [255.08, 3.64])
('preresnet272bn_cifar10', [420.38, 3.25])
('preresnet542bn_cifar10', [833.64, 3.14])
('pyramidnet110_a48_cifar10', [408.37, 3.72])
('pyramidnet110_a84_cifar10', [778.15, 2.98])
('resnet110_cifar10', [255.7, 3.69])
('resnet164bn_cifar10', [255.31, 3.68])
('resnet272bn_cifar10', [420.61, 3.33])
('resnet542bn_cifar10', [833.87, 3.43])
('resnext29_32x4d_cifar10', [780.55, 3.15])
('rir_cifar10', [1281.08, 3.28])
('ror3_164_cifar10', [370.72, 3.93])
('sepreresnet164bn_cifar10', [256.32, 3.73])
('sepreresnet272bn_cifar10', [422.45, 3.39])
('sepreresnet542bn_cifar10', [837.78, 3.08])
('seresnet110_cifar10', [255.98, 3.63])
('seresnet164bn_cifar10', [256.55, 3.39])
('seresnet272bn_cifar10', [422.68

In [ ]:
print(len(model_list))

70


In [ ]:
model_names=sorted(model_names)
for m in model_names:
    print(m)

('densenet100_k12_cifar10', [1353.55, 3.66])
('densenet40_k36_bc_cifar10', [654.6, 4.04])
('diapreresnet164bn_cifar10', [343.37, 3.56])
('diaresnet110_cifar10', [264.71, 4.1])
('diaresnet164bn_cifar10', [343.6, 3.5])
('preresnet110_cifar10', [255.68, 3.86])
('preresnet164bn_cifar10', [255.08, 3.64])
('preresnet272bn_cifar10', [420.38, 3.25])
('preresnet542bn_cifar10', [833.64, 3.14])
('pyramidnet110_a48_cifar10', [408.37, 3.72])
('pyramidnet110_a84_cifar10', [778.15, 2.98])
('resnet110_cifar10', [255.7, 3.69])
('resnet164bn_cifar10', [255.31, 3.68])
('resnet272bn_cifar10', [420.61, 3.33])
('resnet542bn_cifar10', [833.87, 3.43])
('resnext29_32x4d_cifar10', [780.55, 3.15])
('rir_cifar10', [1281.08, 3.28])
('ror3_164_cifar10', [370.72, 3.93])
('sepreresnet164bn_cifar10', [256.32, 3.73])
('sepreresnet272bn_cifar10', [422.45, 3.39])
('sepreresnet542bn_cifar10', [837.78, 3.08])
('seresnet110_cifar10', [255.98, 3.63])
('seresnet164bn_cifar10', [256.55, 3.39])
('seresnet272bn_cifar10', [422.68

In [ ]:
print(model_names)

[('densenet100_k12_cifar10', [1353.55, 3.66]), ('densenet40_k36_bc_cifar10', [654.6, 4.04]), ('diapreresnet164bn_cifar10', [343.37, 3.56]), ('diaresnet110_cifar10', [264.71, 4.1]), ('diaresnet164bn_cifar10', [343.6, 3.5]), ('preresnet110_cifar10', [255.68, 3.86]), ('preresnet164bn_cifar10', [255.08, 3.64]), ('preresnet272bn_cifar10', [420.38, 3.25]), ('preresnet542bn_cifar10', [833.64, 3.14]), ('pyramidnet110_a48_cifar10', [408.37, 3.72]), ('pyramidnet110_a84_cifar10', [778.15, 2.98]), ('resnet110_cifar10', [255.7, 3.69]), ('resnet164bn_cifar10', [255.31, 3.68]), ('resnet272bn_cifar10', [420.61, 3.33]), ('resnet542bn_cifar10', [833.87, 3.43]), ('resnext29_32x4d_cifar10', [780.55, 3.15]), ('rir_cifar10', [1281.08, 3.28]), ('ror3_164_cifar10', [370.72, 3.93]), ('sepreresnet164bn_cifar10', [256.32, 3.73]), ('sepreresnet272bn_cifar10', [422.45, 3.39]), ('sepreresnet542bn_cifar10', [837.78, 3.08]), ('seresnet110_cifar10', [255.98, 3.63]), ('seresnet164bn_cifar10', [256.55, 3.39]), ('seresne

In [ ]:
%cd mifgsm
! ls
%cd ..

/home/tonic/CCW/ml/hw10/mifgsm
airplane  automobile  bird  cat  deer  dog  frog  horse  ship  truck
/home/tonic/CCW/ml/hw10


In [ ]:
!cd ..

In [ ]:
%cd mifgsm
!tar zcvf ../mifgsm.tgz *
%cd ..


/home/tonic/CCW/ml/hw10/mifgsm
airplane/
airplane/airplane19.png
airplane/airplane11.png
airplane/airplane17.png
airplane/airplane2.png
airplane/airplane13.png
airplane/airplane4.png
airplane/airplane3.png
airplane/airplane20.png
airplane/airplane5.png
airplane/airplane10.png
airplane/airplane8.png
airplane/airplane6.png
airplane/airplane12.png
airplane/airplane18.png
airplane/airplane16.png
airplane/airplane7.png
airplane/airplane15.png
airplane/airplane14.png
airplane/airplane9.png
airplane/airplane1.png
automobile/
automobile/automobile12.png
automobile/automobile19.png
automobile/automobile14.png
automobile/automobile1.png
automobile/automobile5.png
automobile/automobile20.png
automobile/automobile16.png
automobile/automobile4.png
automobile/automobile10.png
automobile/automobile7.png
automobile/automobile11.png
automobile/automobile13.png
automobile/automobile3.png
automobile/automobile9.png
automobile/automobile8.png
automobile/automobile15.png
automobile/automobile18.png
automob

In [ ]:
from google.colab import files
files.download('ifgsm.tgz')


ModuleNotFoundError: No module named 'google.colab'

## Example of Ensemble Attack
* Ensemble multiple models as your proxy model to increase the black-box transferability ([paper](https://arxiv.org/abs/1611.02770))

In [ ]:
################ BOSS BASELINE ######################

class ensembleNet(nn.Module):
    def __init__(self, model_names):
        super().__init__()
        self.models = nn.ModuleList([ptcv_get_model(name, pretrained=True) for name in model_names])
        
    def forward(self, x):
        #################### TODO: boss baseline ###################
        ensemble_logits=torch.zeros(10,1)
        for i, m in enumerate(self.models):
          ensemble_logits+=m(x)
          
        # TODO: sum up logits from multiple models  
        return ensemble_logits

* Construct your ensemble model

In [ ]:
model_names = [
    'nin_cifar10',
    'msdnet_cifar10',
    'resnet110_cifar10',
    'preresnet56_cifar10',
    'resnext29_16x64d_cifar10',
]

for model_name in model_names:
  model_checker(model_name)

ensemble_model = ensembleNet(model_names).to(device)
ensemble_model.eval()

## Visualization

In [ ]:
import matplotlib.pyplot as plt

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10, 20))
cnt = 0
for i, cls_name in enumerate(classes):
    path = f'{cls_name}/{cls_name}1.png'
    # benign image
    cnt += 1
    plt.subplot(len(classes), 4, cnt)
    im = Image.open(f'./data/{path}')
    logit = model(transform(im).unsqueeze(0).to(device))[0]
    predict = logit.argmax(-1).item()
    prob = logit.softmax(-1)[predict].item()
    plt.title(f'benign: {cls_name}1.png\n{classes[predict]}: {prob:.2%}')
    plt.axis('off')
    plt.imshow(np.array(im))
    # adversarial image
    cnt += 1
    plt.subplot(len(classes), 4, cnt)
    im = Image.open(f'./fgsm/{path}')
    logit = model(transform(im).unsqueeze(0).to(device))[0]
    predict = logit.argmax(-1).item()
    prob = logit.softmax(-1)[predict].item()
    plt.title(f'adversarial: {cls_name}1.png\n{classes[predict]}: {prob:.2%}')
    plt.axis('off')
    plt.imshow(np.array(im))
plt.tight_layout()
plt.show()

## Report Question
* Make sure you follow below setup: the source model is "resnet110_cifar10", applying the vanilla fgsm attack on `dog2.png`. You can find the perturbed image in `fgsm/dog2.png`.

In [ ]:
# original image
path = f'dog/dog2.png'
im = Image.open(f'./data/{path}')
logit = model(transform(im).unsqueeze(0).to(device))[0]
predict = logit.argmax(-1).item()
prob = logit.softmax(-1)[predict].item()
plt.title(f'benign: dog2.png\n{classes[predict]}: {prob:.2%}')
plt.axis('off')
plt.imshow(np.array(im))
plt.tight_layout()
plt.show()

# adversarial image 
adv_im = Image.open(f'./fgsm/{path}')
logit = model(transform(adv_im).unsqueeze(0).to(device))[0]
predict = logit.argmax(-1).item()
prob = logit.softmax(-1)[predict].item()
plt.title(f'adversarial: dog2.png\n{classes[predict]}: {prob:.2%}')
plt.axis('off')
plt.imshow(np.array(adv_im))
plt.tight_layout()
plt.show()

## Passive Defense - JPEG compression
JPEG compression by imgaug package, compression rate set to 70

Reference: https://imgaug.readthedocs.io/en/latest/source/api_augmenters_arithmetic.html#imgaug.augmenters.arithmetic.JpegCompression

Note: If you haven't implemented the JPEG compression, this module will return an error. Don't worry about this.

In [ ]:
import imgaug.augmenters as iaa

# pre-process image
x = transforms.ToTensor()(adv_im)*255
x = x.permute(1, 2, 0).numpy()
x = x.astype(np.uint8)

# TODO: use "imgaug" package to perform JPEG compression (compression rate = 70)
# compressed_x =  ... x .. 

logit = model(transform(compressed_x).unsqueeze(0).to(device))[0]
predict = logit.argmax(-1).item()
prob = logit.softmax(-1)[predict].item()
plt.title(f'JPEG adversarial: dog2.png\n{classes[predict]}: {prob:.2%}')
plt.axis('off')


plt.imshow(compressed_x)
plt.tight_layout()
plt.show()